In [4]:
import os

In [2]:
!wget https://www.crcv.ucf.edu/data/UCF101/UCF101.rar --no-check-certificate



--2023-10-04 09:33:57--  https://www.crcv.ucf.edu/data/UCF101/UCF101.rar
Resolving www.crcv.ucf.edu (www.crcv.ucf.edu)... 132.170.214.127
Connecting to www.crcv.ucf.edu (www.crcv.ucf.edu)|132.170.214.127|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 6932971618 (6.5G) [application/rar]
Saving to: ‘UCF101.rar’

UCF101.rar          100%[===================>]   6.46G  45.4MB/s    in 2m 42s  

2023-10-04 09:36:39 (40.9 MB/s) - ‘UCF101.rar’ saved [6932971618/6932971618]



In [3]:
!mkdir data

In [4]:


!unrar x /content/UCF101.rar -d data/

Streaming output truncated to the last 5000 lines.
Extracting  data/UCF-101/PlayingGuitar/v_PlayingGuitar_g06_c05.avi        62%  OK 
Extracting  data/UCF-101/PlayingGuitar/v_PlayingGuitar_g06_c06.avi        62%  OK 
Extracting  data/UCF-101/PlayingGuitar/v_PlayingGuitar_g06_c07.avi        62%  OK 
Extracting  data/UCF-101/PlayingGuitar/v_PlayingGuitar_g07_c01.avi        62%  OK 
Extracting  data/UCF-101/PlayingGuitar/v_PlayingGuitar_g07_c02.avi        62%  OK 
Extracting  data/UCF-101/PlayingGuitar/v_PlayingGuitar_g07_c03.avi        62%  OK 
Extracting  data/UCF-101/PlayingGuitar/v_PlayingGuitar_g07_c04.avi        62%  OK 
Extracting  data/UCF-101/PlayingGuitar/v_PlayingGuitar_g07_c05.avi        62%  OK 
Extracting  data/UCF-101/PlayingGuitar/v_PlayingGuitar_g07_c06.avi        62%  OK 
Extracting  data/UCF-101/PlayingGuitar/v_PlayingGuitar_g07_c07.avi        62%  OK 
Extracting  data/UCF-101/Play

In [1]:
import cv2
import numpy as np
import random
import matplotlib.pyplot as plt


In [2]:
data_path = '/content/data/UCF-101'

In [5]:
num_classes = 3
selected_classes = random.sample(os.listdir(data_path), num_classes)
print('Selected classes:', selected_classes)

Selected classes: ['Bowling', 'Skijet', 'Archery']


In [6]:
# Function to extract frames from a video
def extract_frames(video_path, num_frames=16):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_indices = np.linspace(0, total_frames - 1, num_frames, dtype=int)

    frames = []
    for index in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, index)
        ret, frame = cap.read()
        if ret:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # OpenCV loads images in BGR, convert to RGB
            frames.append(frame)

    cap.release()
    return frames

In [7]:
# Function to preprocess a frame
def preprocess_frame(frame, target_shape=(224, 224)):
    # Resize and normalize the frame
    resized_frame = cv2.resize(frame, target_shape)
    normalized_frame = resized_frame.astype(np.float32) / 255.0
    return normalized_frame


In [8]:
# Function to preprocess a video
def preprocess_video(video_path):
    frames = extract_frames(video_path)
    preprocessed_frames = [preprocess_frame(frame) for frame in frames]
    return np.stack(preprocessed_frames)  # Stack frames to create a video


In [9]:
# Prepare the data and labels
train_data = []
train_labels = []

test_data = []
test_labels = []

for class_label, class_name in enumerate(selected_classes):
    class_path = os.path.join(data_path, class_name)
    videos = os.listdir(class_path)
    random.shuffle(videos)
    # Use 80% of the videos for training and 20% for testing
    num_train_videos = int(0.8 * len(videos))
    train_videos = videos[:num_train_videos]
    test_videos = videos[num_train_videos:]

    for video in train_videos:
        video_path = os.path.join(class_path, video)
        train_data.append(preprocess_video(video_path))
        train_labels.append(class_label)

    for video in test_videos:
        video_path = os.path.join(class_path, video)
        test_data.append(preprocess_video(video_path))
        test_labels.append(class_label)
        # Convert to numpy arrays
train_data = np.array(train_data)
train_labels = np.array(train_labels)
test_data = np.array(test_data)
test_labels = np.array(test_labels)

# Print the shapes
print('Train data shape:', train_data.shape)
print('Train labels shape:', train_labels.shape)
print('Test data shape:', test_data.shape)
print('Test labels shape:', test_labels.shape)

Train data shape: (320, 16, 224, 224, 3)
Train labels shape: (320,)
Test data shape: (80, 16, 224, 224, 3)
Test labels shape: (80,)


In [10]:


# Split the training data into training and validation sets
from sklearn.model_selection import train_test_split

# Split the data and labels into training and validation sets
train_data, val_data, train_labels, val_labels = train_test_split(
    train_data, train_labels, test_size=0.2, random_state=42, stratify=train_labels)

# Print the shapes of the sets
print('Train data shape:', train_data.shape)
print('Train labels shape:', train_labels.shape)
print('Validation data shape:', val_data.shape)
print('Validation labels shape:', val_labels.shape)




Train data shape: (256, 16, 224, 224, 3)
Train labels shape: (256,)
Validation data shape: (64, 16, 224, 224, 3)
Validation labels shape: (64,)


In [11]:
import numpy as np
import cv2

# Define a function to preprocess video data
def preprocess_video_data(video_data, target_shape=(224, 224)):
    resized_frames = [cv2.resize(frame, target_shape) for frame in video_data]
    normalized_frames = np.array(resized_frames) / 255.0  # Normalize pixel values
    return normalized_frames

# Preprocess train, validation, and test data
train_data_processed = [preprocess_video_data(video) for video in train_data]
val_data_processed = [preprocess_video_data(video) for video in val_data]
test_data_processed = [preprocess_video_data(video) for video in test_data]

# Convert lists to numpy arrays
train_data_processed = np.array(train_data_processed)
val_data_processed = np.array(val_data_processed)
test_data_processed = np.array(test_data_processed)


In [ ]:
import matplotlib.pyplot as plt

# Function to plot images and captions
def plot_images_with_captions(images, captions):
    fig, axes = plt.subplots(1, len(images), figsize=(16, 3))
    for i in range(len(images)):
        axes[i].imshow(images[i])
        axes[i].set_title('Caption: ' + captions[i])
        axes[i].axis('off')
    plt.show()

# Assuming you have captions for the frames
captions = ['Caption 1', 'Caption 2']  # Replace with your actual captions

# Plot the frames with captions
plot_images_with_captions(train_data_processed[:2], captions)


In [14]:
train_labels = np.array(train_labels)
val_labels = np.array(val_labels)
test_labels = np.array(test_labels)


In [15]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Dropout, Reshape, GlobalAveragePooling2D
from tensorflow.keras.applications import ResNet50

# Load a pretrained CNN model (e.g., ResNet50) as the encoder
base_cnn_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the pretrained model
base_cnn_model.trainable = False

# Build the CNN encoder
def build_cnn_encoder(base_model):
    inputs = tf.keras.layers.Input(shape=(224, 224, 3))
    x = base_model(inputs, training=False)
    x = GlobalAveragePooling2D()(x)  # Global average pooling
    cnn_encoder = tf.keras.Model(inputs, x, name='cnn_encoder')
    return cnn_encoder

# Build the CNN encoder
cnn_encoder = build_cnn_encoder(base_cnn_model)


In [ ]:
cnn_feature_dim = cnn_encoder.layers[-1].output_shape[-1]

Dropout Layer Justification:
We add dropout layers to prevent overfitting by randomly setting a fraction of the input units to 0 during each update, which helps generalize the model.

Activation Function:
We'll use the softmax activation function for the output layer since we're performing multi-class classification.

Choice of Layers, Activation Function, and Hyperparameters:
We've chosen a 2-layered LSTM model with 128 units, a dropout rate of 0.5 to prevent overfitting, and a softmax activation function for classification. These choices were made based on typical configurations for similar tasks, but fine-tuning may be needed based on the specific dataset and problem.

In [17]:
def build_lstm_model(input_shape, num_lstm_layers=2, lstm_units=128, dropout_rate=0.5, num_classes=3):
    model = tf.keras.Sequential()

    # Reshape the CNN encoder output
    model.add(Reshape((1, input_shape), input_shape=(input_shape,)))

    # LSTM layers
    for i in range(num_lstm_layers):
        return_sequences = (i < num_lstm_layers - 1)
        model.add(LSTM(lstm_units, return_sequences=return_sequences))
        model.add(Dropout(dropout_rate))

    # Dense layer for classification
    model.add(Dense(num_classes, activation='softmax'))

    return model

# Assuming cnn_feature_dim is the dimension of the CNN encoder output
cnn_feature_dim = 2048  # Adjust based on the chosen pretrained model's output dimension

# Example: Build a 2-layered LSTM model
num_lstm_layers = 2
lstm_units = 128
dropout_rate = 0.5
num_classes = 3  # Adjust this based on the chosen number of classes

# Build the LSTM model
lstm_model = build_lstm_model(cnn_feature_dim, num_lstm_layers, lstm_units, dropout_rate, num_classes)

# Print the model summary
lstm_model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 1, 2048)           0         
                                                                 
 lstm (LSTM)                 (None, 1, 128)            1114624   
                                                                 
 dropout (Dropout)           (None, 1, 128)            0         
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense (Dense)               (None, 3)                 387       
                                                                 
Total params: 1246595 (4.76 MB)
Trainable params: 124659

Model Compilation:
Loss Function, Optimizer, and Learning Rate Choice:
We'll use categorical crossentropy as the loss function (suitable for multi-class classification), the Adam optimizer for optimization, and a learning rate of 0.001.

Justification:
Categorical crossentropy is commonly used for multi-class classification. Adam optimizer with a learning rate of 0.001 is a popular choice due to its adaptive learning rate properties.

In [18]:

# Compile the model
lstm_model.compile(loss='categorical_crossentropy',
                   optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                   metrics=['accuracy'])



Model Training:
Train the Model, Print Train and Validation Loss, Plot History:

In [ ]:
import matplotlib.pyplot as plt

# Assuming you have train_data, train_labels, val_data, val_labels

# Define training parameters
batch_size = 32
epochs = 10  # Adjust the number of epochs as needed

# Train the model
history = lstm_model.fit(train_data, train_labels,
                         validation_data=(val_data, val_labels),
                         epochs=epochs, batch_size=batch_size)

# Print train and validation loss for each epoch
print('Train Loss:', history.history['loss'])
print('Validation Loss:', history.history['val_loss'])

# Plot the loss and accuracy history graphs
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

# Print the total time taken for training
total_training_time = sum(history.epoch)
print('Total time taken for training:', total_training_time)




In [ ]:
import random
from sklearn.metrics import confusion_matrix, classification_report

# Assuming you have test_data and test_labels

# Take 5 random samples from the test set for evaluation
random_indices = random.sample(range(len(test_data)), 5)
random_test_data = [test_data[i] for i in random_indices]
random_test_labels = [test_labels[i] for i in random_indices]

# Perform expression recognition on the random test data
predictions = lstm_model.predict(np.array(random_test_data))

# Print confusion metrics and classification report
print('Confusion Matrix:')
print(confusion_matrix(np.argmax(random_test_labels, axis=1), np.argmax(predictions, axis=1)))

print('\nClassification Report:')
print(classification_report(np.argmax(random_test_labels, axis=1), np.argmax(predictions, axis=1)))
